In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import cudf
import cupy
import nvtabular as nvt
import json

/opt/conda/lib/python3.8/site-packages/tqdm/std.py:725: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
DATA_FOLDER = '/Users/Danie/Downloads/SIGIR-ecom-data-challenge/train'
FILENAME_PATTERN_BROWSING = 'browsing_train.csv'
FILENAME_PATTERN_SEARCH = 'search_train.csv'
DATA_PATH_BROWSING = os.path.join(DATA_FOLDER, FILENAME_PATTERN_BROWSING)
DATA_PATH_SEARCH = os.path.join(DATA_FOLDER, FILENAME_PATTERN_SEARCH)
OUTPUT_DIR = "/home/dbanks/recodata"


In [3]:
DATA_PATH_SEARCH

'/Users/Danie/Downloads/SIGIR-ecom-data-challenge/train/search_train.csv'

In [4]:
MINIMUM_SESSION_LENGTH = 2

In [5]:
# load search data
search = pd.read_csv('search_train.csv')
# Add column event_type 
search['event_type'] = 'search'
# Add column 'is_search'
search['is_search'] = 1
# drop 123 rows where: (clicked_skus_hash != NaN) and (product_skus_hash == NaN)
condition = (search['product_skus_hash'].isnull()) & (~search['clicked_skus_hash'].isnull())
search = search.loc[~condition]
# convert strings to list object 
import ast
def convert_str_to_list(x): 
    if pd.isnull(x): 
        return x
    return ast.literal_eval(x)
for col in ['product_skus_hash', 'clicked_skus_hash', 'query_vector']: 
    search[col] = search[col].progress_apply(convert_str_to_list)



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 819393/819393 [02:04<00:00, 6556.18it/s]


In [6]:
tmp = search.groupby('session_id_hash').size().reset_index()
tmp.columns = ['session_id_hash', 'nb_queries']
search = search.merge(tmp, on='session_id_hash', how='left')

Include unseen clicked product to impression list :

2% of the search events have a clicked item that does not appear in the impression list.

  ==> We add the missing elements to the impression list.

In [7]:
def add_clicked(x): 
    if isinstance(x.clicked_skus_hash, list) and isinstance(x.product_skus_hash, list):
        return list(set(x.product_skus_hash).union(set(x.clicked_skus_hash)))
    return x.product_skus_hash
search['updated_product_skus_hash'] = search.progress_apply(add_clicked, axis=1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 819393/819393 [00:20<00:00, 39912.92it/s]


Compute the number of visualized products and clicked items per search

In [8]:
search['impression_size'] = search.product_skus_hash.str.len().fillna(0)
search['clicks_size'] = search.clicked_skus_hash.str.len().fillna(0)

In [9]:
search.head(5)


,session_id_hash,query_vector,clicked_skus_hash,product_skus_hash,server_timestamp_epoch_ms,event_type,is_search,nb_queries,updated_product_skus_hash,impression_size,clicks_size
0,48fade624d47870058ce07dd789ccc04e46c70c0fa2a1b...,"[-0.20255649089813232, -0.016908567398786545, ...",NaN,NaN,1548575194779,search,1,1,NaN,0.0,0.0
1,8731ca84ff7bb8cb647531d54e64feedb2519b4a7792a7...,"[-0.007610442116856575, -0.14909175038337708, ...",NaN,[9ee9ffd7e2529a65f9a0b0c9eaae6330df85cf2e3af33...,1548276763869,search,1,1,[9ee9ffd7e2529a65f9a0b0c9eaae6330df85cf2e3af33...,3.0,0.0
2,9be980708345944960645d03606ea83b637cae9106b705...,"[-0.20023074746131897, -0.03151938319206238, 0...",NaN,[7cc72dbed53bab78ec6a62feaa5052a7a1db7d201664b...,1548937997295,search,1,3,[7cc72dbed53bab78ec6a62feaa5052a7a1db7d201664b...,7.0,0.0
3,9be980708345944960645d03606ea83b637cae9106b705...,"[-0.18556387722492218, -0.07620412111282349, 0...",NaN,[62c4ddab6c1c81c74d315376b3c0dc7768c0286b3dc6f...,1548938038268,search,1,3,[62c4ddab6c1c81c74d315376b3c0dc7768c0286b3dc6f...,13.0,0.0
4,9be980708345944960645d03606ea83b637cae9106b705...,"[-0.03269264101982117, -0.27234694361686707, 0...",NaN,[2a0ee2924feabeec35e21e8fcb4d5b0684d190e46cef7...,1548938093827,search,1,3,[2a0ee2924feabeec35e21e8fcb4d5b0684d190e46cef7...,25.0,0.0


Aggregate search information per session

The resulting variables are:

- "flat_query_vector": List of search query vector (reshaped to 1-dim so that it works with NVT Data loader)
- "flat_product_skus_hash": List of product skus returned by the search query
- "clicked-flag": To indicate if the product was clicked or not in the search results
- "nb_queries" : number of search queries within the same session 
- "impression_size": number of returned product skus for each search query
- "clicks_size": number of clicked items for each search query

In [10]:
def all_products(x): 
    t =[]
    for products in x.dropna(): 
        t += products
    if len(t)==0:
        return ['missing']
    return t

session_search = search.sort_values(['session_id_hash', 'server_timestamp_epoch_ms']).groupby('session_id_hash').agg({'query_vector': lambda x: list(np.concatenate(x.values)),
                                                                                                                    'updated_product_skus_hash': all_products,
                                                                                                                    'clicked_skus_hash': all_products,
                                                                                                                    'impression_size': list,
                                                                                                                    'clicks_size': list,
                                                                                                                    'nb_queries': 'last'
                                                                                                                  })
session_search.columns = ['flat_query_vector', 'flat_product_skus_hash', 'flat_clicked_skus_hash', 'impressions_size', 'clicks_size', 'nb_queries']
session_search['clicked-flag'] = session_search.progress_apply(lambda x: [int(e in x['flat_clicked_skus_hash']) for e in x['flat_product_skus_hash']], axis=1)
session_search = session_search.reset_index()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 550028/550028 [00:47<00:00, 11678.58it/s]


In [11]:
session_search.head(3)

,session_id_hash,flat_query_vector,flat_product_skus_hash,flat_clicked_skus_hash,impressions_size,clicks_size,nb_queries,clicked-flag
0,00000277639fc5c6f816654b78bf3654ece7fd53a7338f...,"[-0.1088152825832367, -0.09937708079814911, -0...",[5c05011d4376242ed29a50d25215abccae158b49bbfb1...,[ca089db21669aabe32904308205ad3632574fd268e912...,[14.0],[1.0],1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
1,00001216514808fdd1e3bd4e4229ef518c712932424d82...,"[-0.05604613572359085, -0.19420677423477173, 0...",[e90aeee7e8eb921bd430981037ee19c6e95226d0651c6...,[missing],[25.0],[0.0],1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,000049b258d564d8a44105a5c85b1a82e69bae9129319b...,"[-0.018799802288413048, -0.2817137837409973, 0...",[7302590bebd27fb6dece6dc605b96a098ab1efdf35aff...,[missing],"[0.0, 21.0]","[0.0, 0.0]",2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [12]:
session_search.to_parquet(os.path.join(OUTPUT_DIR, "session_search.parquet"))
search.to_parquet(os.path.join(OUTPUT_DIR, "search.parquet"))

FileNotFoundError: [Errno 2] No such file or directory: '/home/dbanks/recodata/session_search.parquet'

## Create event table:

In [13]:
session_search = pd.read_parquet(os.path.join(OUTPUT_DIR, "session_search.parquet"))
search = pd.read_parquet(os.path.join(OUTPUT_DIR, "search.parquet"))

FileNotFoundError: /home/dbanks/recodata/session_search.parquet

In [14]:
browsing = pd.read_csv('browsing_train.csv')
# Add columns 'is_search'
browsing['is_search'] = 0 


In [15]:

event_df = browsing
#event_df.reset_index(drop=True, inplace=True)

In [16]:
event_df.head(3)

,session_id_hash,event_type,product_action,product_sku_hash,server_timestamp_epoch_ms,hashed_url,is_search
0,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,d5157f8bc52965390fa21ad5842a8502bc3eb8b0930f3f...,1550885210881,7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...,0
1,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,61ef3869355b78e11011f39fc7ac8f8dfb209b3442a9d5...,1550885213307,4ed279f4f0deab6dfc80f4f7bf49d527fd894fa478a9ce...,0
2,20c458b802f6ea9374783bfc528b19421be977a6769785...,pageview,NaN,NaN,1550885213307,4ed279f4f0deab6dfc80f4f7bf49d527fd894fa478a9ce...,0


In [17]:
#del browsing

In [18]:
# drop duplicated rows (total of 1800) 
event_df.drop_duplicates(['session_id_hash' ,'server_timestamp_epoch_ms', 'event_type', 'product_action', 'product_sku_hash', 'hashed_url'],inplace=True, keep='last')
# Remove indices of 'pageview' interactions from duplicated events defined above 
tmp = event_df[(event_df.event_type == 'pageview') & (event_df.duplicated(['session_id_hash' , 'server_timestamp_epoch_ms']))]
event_df.drop(tmp.index, inplace=True)

In [19]:
event_df.shape

(31192753, 7)

In [20]:
# select search events with clicks
use_cols = ['session_id_hash', 'clicked_skus_hash',
            'server_timestamp_epoch_ms', 'event_type',
            'is_search']
search_clicks = search[search.clicks_size>0][use_cols]

# specify event type and product action
search_clicks['event_type'] = 'search'
search_clicks['product_action'] = 'click'

#unstack list of clicked items to multiple rows
lst_col = 'clicked_skus_hash'
search_clicks = pd.DataFrame({
    col:np.repeat(search_clicks[col].values, search_clicks[lst_col].str.len()) for col in search_clicks.columns.difference([lst_col])}).assign(
    **{lst_col:np.concatenate(search_clicks[lst_col].values)})[search_clicks.columns.tolist()]

search_clicks.columns = ['session_id_hash', 'product_sku_hash', 'server_timestamp_epoch_ms',
                         'event_type', 'is_search', 'product_action']

In [21]:
search_clicks.head(3)

,session_id_hash,product_sku_hash,server_timestamp_epoch_ms,event_type,is_search,product_action
0,77d604ce112a107ffe3e60566778d090a0b756bcfb0a3e...,20e8f1c4051a6c55122bd00f25f92f91b5f125497b7939...,1549078957672,search,1,click
1,7fc129b5945b2959339da1c0675d9c9adcdbc1fc99a954...,8283e8b59c283c3b72462a6f73084683524f4b0149acb7...,1550003058242,search,1,click
2,3ef3f3170f85ae92b01378b2161f5e71f64caadbeefd90...,2901d00c55fc2e4e60ea0d0e7a48ee843491e31025afab...,1549641526668,search,1,click


In [22]:
event_df = pd.concat([event_df, search_clicks])
event_df.event_type.value_counts()

pageview         20762222
event_product    10430531
search             391820
Name: event_type, dtype: int64

In [23]:
product_info = pd.read_csv('sku_to_content.csv')
product_info.tail()

,product_sku_hash,description_vector,category_hash,image_vector,price_bucket
66381,c7cc673ca3baa5fa222fffdc16379892b3a62583a48143...,NaN,NaN,NaN,NaN
66382,6641c7d2053ce48ce1e81a9653dffe56dbb79ab0704fbd...,"[-0.19150441884994507, -0.06235162168741226, -...",0665a81d19c89281cc00e7f7d779ded2ed42c933838602...,"[129.55668732976045, 43.27996741934932, -36.70...",3.0
66383,526a6a51717d5bb40ef2b0c47394d08c54385375633bab...,"[-0.19760936498641968, 0.4446450471878052, -0....",0665a81d19c89281cc00e7f7d779ded2ed42c933838602...,"[-158.25984189321855, 74.19255741438077, 199.6...",5.0
66384,21ca4ab0e2fbd3b401fbeadeb4439dcab9998fb52159ec...,NaN,NaN,NaN,NaN
66385,0eaa39fb645749da181c0637d3b420f3f103b5f6b50286...,NaN,NaN,NaN,NaN


Extract product main category

In [24]:
def product_main_category(x):
    if pd.isna(x):
        return x
    return x.split('/')[0]
product_info['main_category'] = product_info['category_hash'].progress_apply(product_main_category)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 66386/66386 [00:00<00:00, 474366.32it/s]


Compute average price of main and hierarchy category

In [25]:
main_price = product_info.groupby('main_category')['price_bucket'].mean().reset_index()
main_price.columns = ['main_category', 'mean_price_main']

hierarchy_price = product_info.groupby('category_hash')['price_bucket'].mean().reset_index()
hierarchy_price.columns = ['category_hash', 'mean_price_hierarchy']

Merge with product table

In [26]:
product_info = product_info.merge(main_price, on=['main_category'], how='left')
product_info = product_info.merge(hierarchy_price, on=['category_hash'], how='left')

In [27]:
product_info.head(3)

,product_sku_hash,description_vector,category_hash,image_vector,price_bucket,main_category,mean_price_main,mean_price_hierarchy
0,26ce7b47f4c46e4087e83e54d2f7ddc7ea57862fed2e2a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6383992be772b204a9ab75f86c86f5583d1bdd1222952d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a2c3e2430c6ef9770b903ad08fa067a6b2b9db28f06e1b...,"[0.27629122138023376, -0.15763211250305176, 0....",06fa312761d4b39e2f649781514ac69a4c1505c221fc46...,"[340.3592564184389, -220.19025864725685, 154.0...",7.0,06fa312761d4b39e2f649781514ac69a4c1505c221fc46...,5.69263,7.859416


Merge product table and event_df

In [28]:
event_df = event_df.merge(product_info[['product_sku_hash', 'main_category', 'category_hash',
                                        'price_bucket', 'mean_price_hierarchy', 'mean_price_main' ]], on='product_sku_hash', how='left')

In [29]:
event_df.head(2)

event_df.shape


for col in event_df.columns:
    print(col)

session_id_hash
event_type
product_action
product_sku_hash
server_timestamp_epoch_ms
hashed_url
is_search
main_category
category_hash
price_bucket
mean_price_hierarchy
mean_price_main


Remove duplicated interactions with the same product: encode original information as binary flags

In [30]:
product_interactions = event_df.groupby(['session_id_hash', 'product_sku_hash']).agg({
    'product_sku_hash': len,
    'product_action': [lambda x: 'add' in list(x), 
                       lambda x: 'detail' in list(x),
                       lambda x: 'purchase' in list(x),
                       lambda x: 'remove' in list(x),
                       lambda x: 'click' in list(x),
                      ]
}).droplevel(0, axis=1)

In [31]:
product_interactions = product_interactions.reset_index()

In [32]:
product_interactions.columns = ['session_id_hash', 'product_sku_hash', 'nb_interactions',
                                'has_been_added_to_cart', 'has_been_detailed', 
                                'has_been_purchased', 'has_been_removed_from_cart', 'has_been_clicked' ]

In [33]:

product_interactions.head(2)

,session_id_hash,product_sku_hash,nb_interactions,has_been_added_to_cart,has_been_detailed,has_been_purchased,has_been_removed_from_cart,has_been_clicked
0,00000114e1075962f022114fcfc17f2d874e694ac5d201...,4945f2fa8e87cb7501702ed3dce26253296eae7a8f670f...,1,False,True,False,False,False
1,00000114e1075962f022114fcfc17f2d874e694ac5d201...,6ff8d0f30bbe66cfec7d87fc7e22bd8b1defd47ff4aaaa...,1,False,True,False,False,False


Merge product table and event_df

In [34]:
event_df = event_df.merge(product_info[['product_sku_hash', 'main_category', 'category_hash',
                                        'price_bucket', 'mean_price_hierarchy', 'mean_price_main' ]], on='product_sku_hash', how='left')

In [35]:
event_df.head(2)

,session_id_hash,event_type,product_action,product_sku_hash,server_timestamp_epoch_ms,hashed_url,is_search,main_category_x,category_hash_x,price_bucket_x,mean_price_hierarchy_x,mean_price_main_x,main_category_y,category_hash_y,price_bucket_y,mean_price_hierarchy_y,mean_price_main_y
0,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,d5157f8bc52965390fa21ad5842a8502bc3eb8b0930f3f...,1550885210881,7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...,0,0665a81d19c89281cc00e7f7d779ded2ed42c933838602...,0665a81d19c89281cc00e7f7d779ded2ed42c933838602...,10.0,9.857143,5.009099,0665a81d19c89281cc00e7f7d779ded2ed42c933838602...,0665a81d19c89281cc00e7f7d779ded2ed42c933838602...,10.0,9.857143,5.009099
1,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,61ef3869355b78e11011f39fc7ac8f8dfb209b3442a9d5...,1550885213307,4ed279f4f0deab6dfc80f4f7bf49d527fd894fa478a9ce...,0,0665a81d19c89281cc00e7f7d779ded2ed42c933838602...,0665a81d19c89281cc00e7f7d779ded2ed42c933838602...,10.0,9.857143,5.009099,0665a81d19c89281cc00e7f7d779ded2ed42c933838602...,0665a81d19c89281cc00e7f7d779ded2ed42c933838602...,10.0,9.857143,5.009099


In [36]:
event_df = event_df.sort_values(['session_id_hash', 'server_timestamp_epoch_ms'])

In [37]:
event_df.shape

(31584573, 17)

Remove duplicated interactions with the same product: encode original information as binary flags

In [38]:
event_df = event_df.drop_duplicates(['session_id_hash','event_type','product_sku_hash','hashed_url'], keep='first')

Remove repeated interactions of event-product to align with private test set

In [39]:
event_df.head()

,session_id_hash,event_type,product_action,product_sku_hash,server_timestamp_epoch_ms,hashed_url,is_search,main_category_x,category_hash_x,price_bucket_x,mean_price_hierarchy_x,mean_price_main_x,main_category_y,category_hash_y,price_bucket_y,mean_price_hierarchy_y,mean_price_main_y
20465725,00000114e1075962f022114fcfc17f2d874e694ac5d201...,pageview,NaN,NaN,1552423391039,0aa1084eddfb08e4dffbb5a2aa98a5e9679382d982dd97...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20465726,00000114e1075962f022114fcfc17f2d874e694ac5d201...,event_product,detail,cf2f88cb43c1713538f7dfd2aa498a2cb9ebc0c99feeac...,1552423391039,0aa1084eddfb08e4dffbb5a2aa98a5e9679382d982dd97...,0,06fa312761d4b39e2f649781514ac69a4c1505c221fc46...,06fa312761d4b39e2f649781514ac69a4c1505c221fc46...,10.0,9.469945,5.69263,06fa312761d4b39e2f649781514ac69a4c1505c221fc46...,06fa312761d4b39e2f649781514ac69a4c1505c221fc46...,10.0,9.469945,5.69263
20465727,00000114e1075962f022114fcfc17f2d874e694ac5d201...,event_product,add,cf2f88cb43c1713538f7dfd2aa498a2cb9ebc0c99feeac...,1552424389158,83b4fdad686c1be4eba335f70d23ae202b84b6153e109e...,0,06fa312761d4b39e2f649781514ac69a4c1505c221fc46...,06fa312761d4b39e2f649781514ac69a4c1505c221fc46...,10.0,9.469945,5.69263,06fa312761d4b39e2f649781514ac69a4c1505c221fc46...,06fa312761d4b39e2f649781514ac69a4c1505c221fc46...,10.0,9.469945,5.69263
20465728,00000114e1075962f022114fcfc17f2d874e694ac5d201...,pageview,NaN,NaN,1552424395594,0ad6fab1eb3ac76010ea2fa6399a4e993b00f6501c88a2...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20465729,00000114e1075962f022114fcfc17f2d874e694ac5d201...,pageview,NaN,NaN,1552424417587,e93e5c83aab0987e41d8fd65a30b54d2ce87491b4a7f9b...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
event_df = event_df.drop(columns = ["price_bucket_x",'category_hash_x','main_category_x','main_category_y','category_hash_y','price_bucket_y','mean_price_hierarchy_y','mean_price_main_y'])

In [41]:
event_df = event_df.drop(columns = ["mean_price_hierarchy_x","mean_price_main_x"])

In [42]:
event_df.head()

,session_id_hash,event_type,product_action,product_sku_hash,server_timestamp_epoch_ms,hashed_url,is_search
20465725,00000114e1075962f022114fcfc17f2d874e694ac5d201...,pageview,NaN,NaN,1552423391039,0aa1084eddfb08e4dffbb5a2aa98a5e9679382d982dd97...,0
20465726,00000114e1075962f022114fcfc17f2d874e694ac5d201...,event_product,detail,cf2f88cb43c1713538f7dfd2aa498a2cb9ebc0c99feeac...,1552423391039,0aa1084eddfb08e4dffbb5a2aa98a5e9679382d982dd97...,0
20465727,00000114e1075962f022114fcfc17f2d874e694ac5d201...,event_product,add,cf2f88cb43c1713538f7dfd2aa498a2cb9ebc0c99feeac...,1552424389158,83b4fdad686c1be4eba335f70d23ae202b84b6153e109e...,0
20465728,00000114e1075962f022114fcfc17f2d874e694ac5d201...,pageview,NaN,NaN,1552424395594,0ad6fab1eb3ac76010ea2fa6399a4e993b00f6501c88a2...,0
20465729,00000114e1075962f022114fcfc17f2d874e694ac5d201...,pageview,NaN,NaN,1552424417587,e93e5c83aab0987e41d8fd65a30b54d2ce87491b4a7f9b...,0


Merge product interactions info and event df

In [43]:
event_df = event_df.merge(product_interactions, on=['session_id_hash', 'product_sku_hash'], how='left')

In [44]:
event_df.shape

(21919094, 13)

In [45]:
event_df.head(2)

,session_id_hash,event_type,product_action,product_sku_hash,server_timestamp_epoch_ms,hashed_url,is_search,nb_interactions,has_been_added_to_cart,has_been_detailed,has_been_purchased,has_been_removed_from_cart,has_been_clicked
0,00000114e1075962f022114fcfc17f2d874e694ac5d201...,pageview,NaN,NaN,1552423391039,0aa1084eddfb08e4dffbb5a2aa98a5e9679382d982dd97...,0,NaN,NaN,NaN,NaN,NaN,NaN
1,00000114e1075962f022114fcfc17f2d874e694ac5d201...,event_product,detail,cf2f88cb43c1713538f7dfd2aa498a2cb9ebc0c99feeac...,1552423391039,0aa1084eddfb08e4dffbb5a2aa98a5e9679382d982dd97...,0,3.0,True,True,False,False,False


Add column product_url_hash: SKUs are filled with hashed_url for pageview events

In [46]:
event_df['product_url_hash'] = event_df['product_sku_hash'].fillna(event_df['hashed_url'])

In [47]:
event_df.event_type.value_counts()

pageview         13981064
event_product     7684646
search             253384
Name: event_type, dtype: int64

In [48]:
# save to parquet file with 40 partitions 
event_df['parquet_split'] =  np.random.randint(0,40,size=event_df.shape[0])
event_df.to_parquet(os.path.join(OUTPUT_DIR, "event_train_v4"), partition_cols=['parquet_split'])

In [49]:
import gc
#del event_df
gc.collect()

23

In [ ]:
#event_df.to_csv( "event_sesh1")

## Define the preprocessed row interactions table

1st workflow: fill missing values, encode categorical variables and normalize numericals

In [50]:
import glob
files = glob.glob(OUTPUT_DIR + '/event_train_v4/parquet_split*/*.parquet')

In [51]:
#  load data 
df_event = nvt.Dataset(files, part_size="1GB")  

# convert timestamp to datetime object
to_datetime = ["server_timestamp_epoch_ms"] >> nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col, unit='ms')) >> nvt.ops.Rename( f = lambda x: 'timestamp')

#categorical features
# fill missing product_actions
missing_action = ['product_action'] >> nvt.ops.FillMissing(fill_val='view')  >> nvt.ops.Rename(postfix = '_filled')
categ_feats = ['session_id_hash',  'event_type', 'price_bucket', 'main_category', 'category_hash']
cat_feats = missing_action + categ_feats >> nvt.ops.Categorify()

# fill missing product ids before categorify to keep id '0' for padding 
missing_ids = ['product_sku_hash','hashed_url', 'product_url_hash' ] >> nvt.ops.FillMissing(fill_val='missing')
cat_product_ids = missing_ids >> nvt.ops.Categorify()

# numerical features 
cont_feats = ['mean_price_hierarchy', 'mean_price_main', 'nb_interactions'] >> nvt.ops.FillMedian()
continuous_feats = cont_feats >> nvt.ops.Normalize()

#bool features 
bool_feats = ['has_been_added_to_cart', 'has_been_detailed', 'has_been_purchased', 
              'has_been_removed_from_cart', 'has_been_clicked', 'is_search']
boolean_feats = bool_feats >> nvt.ops.FillMissing(fill_val=0)

RuntimeError: CUDA error encountered at: /workspace/build-env/cpp/src/io/utilities/hostdevice_vector.hpp:60: 2 cudaErrorMemoryAllocation out of memory

In [ ]:
workflow = nvt.Workflow(to_datetime + cat_feats + cat_product_ids + continuous_feats + boolean_feats)
workflow.fit(df_event)
new_gdf = workflow.transform(df_event).to_ddf().compute()
len(new_gdf)

In [ ]:
new_gdf.columns

In [ ]:
workflow.save(os.path.join(OUTPUT_DIR, "categorify_workflow"))

Item recency: include the item first time seen feature using product_url_hash column

In [ ]:
items_first_ts_df = new_gdf.groupby('product_url_hash').agg({'timestamp': 'min'}).reset_index().rename(columns={'timestamp': 'itemid_ts_first'})
interactions_merged_df = new_gdf.merge(items_first_ts_df, on=['product_url_hash'], how='left')

In [ ]:
interactions_merged_df.shape

In [ ]:
use_cols = ['session_id_hash', 'timestamp',  'event_type', 'product_action_filled',
            
            'product_sku_hash','hashed_url', 'product_url_hash',
            
            'main_category', 'category_hash', 'price_bucket', 'mean_price_hierarchy', 'mean_price_main', 'itemid_ts_first', 
            
            'nb_interactions', 'has_been_added_to_cart', 'has_been_detailed', 'has_been_purchased', 'has_been_removed_from_cart', 'has_been_clicked',
            
            'is_search']

interactions_merged_df[use_cols].head(5)

In [ ]:
interactions_merged_df[use_cols].to_parquet(os.path.join(OUTPUT_DIR, 'row_interactions_preproc_v4.parquet'))

In [ ]:
interactions_merged_df.to_csv("interaction_merge.csv")

## Preprocessing of session table

In [ ]:
interactions_merged_df = cudf.read_parquet(os.path.join(OUTPUT_DIR, 'row_interactions_preproc_v4.parquet'))

In [ ]:
from sklearn.model_selection import GroupKFold
sessions_gdf['fold'] = np.random.randint(1,6, sessions_gdf.shape[0]) 
sessions_gdf['is_valid'] = 0 
sessions_gdf.loc[((sessions_gdf['is_test-last']==0) & (sessions_gdf['day_index']<=50)), 'is_valid'] = 1